<a href="https://colab.research.google.com/github/kurikinton105/GDL_code_colab/blob/main/07_02_lstm_compose_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compose: Training a model to generate music

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os
import sys
#sys.path.append('/content/drive/My Drive/Colab Notebooks/GDL/')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/Colab Notebooks/GDL/

/content/drive/My Drive/Colab Notebooks/GDL


In [ ]:
!ls

 02_01_deep_learning_deep_neural_network2.ipynb
 02_01_deep_learning_deep_neural_networkのコピー.ipynb
 02_02_deep_learning_convolutions.ipynb
 02_03_deep_learning_conv_neural_network.ipynb
 03_01_autoencoder_train.ipynb
 03_02_autoencoder_analysis.ipynb
 03_03_vae_digits_train.ipynb
 03_04_vae_digits_analysis.ipynb
 03_05_vae_faces_train.ipynb
 03_06_vae_faces_analysis.ipynb
 04_01_gan_camel_train.ipynb
 04_02_wgan_cifar_train.ipynb
 04_03_wgangp_faces_train.ipynb
 05_01_cyclegan_train.ipynb
 06_01_lstm_text_train.ipynb
 06_02_qa_train.ipynb
 06_03_qa_analysis.ipynb
 07_01_notation_compose.ipynb
 07_02_lstm_compose_train.ipynb
 07_03_lstm_compose_predict.ipynb
 07_04_musegan_train.ipynb
 07_05_musegan_analysis.ipynb
 09_01_positional_encoding.ipynb
'beTF2.0.0_2_01_deep_learning_deep_neural_network.ipynb のコピー'
 data
 Dockerfile.cpu
 Dockerfile.gpu
 keras
 launch-docker-cpu.sh
 launch-docker-gpu.sh
 LICENSE
 model.png
 models
 README.md
 requirements.txt
 results
 run
 scripts
'TF1.15.2_0

In [ ]:
import os
import pickle
import numpy
from music21 import note, chord

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network
base = "/content/drive/My Drive/Colab Notebooks/GDL/"

## Set parameters

In [ ]:
# run params
section = 'compose'
run_id = '0006'
music_name = 'mirai'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = base +os.path.join('data', music_name)
print(data_folder)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

/content/drive/My Drive/Colab Notebooks/GDL/data/mirai


## Extract the notes

In [ ]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f) 

1 files in total
1 Parsing /content/drive/My Drive/Colab Notebooks/GDL/data/mirai/mirai.mid


## Create the lookup tables

In [ ]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [ ]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A4': 0,
 'B-4': 1,
 'B4': 2,
 'C#4': 3,
 'C#5': 4,
 'C5': 5,
 'D4': 6,
 'D5': 7,
 'E4': 8,
 'E5': 9,
 'F#4': 10,
 'F#5': 11,
 'G#4': 12,
 'G4': 13,
 'START': 14}

In [ ]:
print('\nduration_to_int')
duration_to_int


duration_to_int


{0: 0,
 0.25: 1,
 0.5: 2,
 0.75: 3,
 1.0: 4,
 1.25: 5,
 1.5: 6,
 2.0: 7,
 2.75: 8,
 Fraction(10, 3): 9,
 3.75: 10}

## Prepare the sequences used by the Neural Network

In [ ]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [ ]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
duration output
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Create the structure of the neural network

In [ ]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    1500        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1100        input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
plot_model(model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)

AttributeError: ignored

## Train the neural network

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



Epoch 1/2000000
9/9 [==============================] - 3s 385ms/step - loss: 4.1475 - pitch_loss: 2.4630 - duration_loss: 1.6844 - val_loss: 4.0132 - val_pitch_loss: 2.6704 - val_duration_loss: 1.3428
Epoch 2/2000000
9/9 [==============================] - 3s 298ms/step - loss: 3.6596 - pitch_loss: 2.2772 - duration_loss: 1.3824 - val_loss: 4.0527 - val_pitch_loss: 2.6493 - val_duration_loss: 1.4035
Epoch 3/2000000
9/9 [==============================] - 3s 301ms/step - loss: 3.6496 - pitch_loss: 2.2642 - duration_loss: 1.3854 - val_loss: 3.8367 - val_pitch_loss: 2.4869 - val_duration_loss: 1.3499
Epoch 4/2000000
9/9 [==============================] - 3s 289ms/step - loss: 3.6311 - pitch_loss: 2.2576 - duration_loss: 1.3735 - val_loss: 3.8691 - val_pitch_loss: 2.5308 - val_duration_loss: 1.3383
Epoch 5/2000000
9/9 [==============================] - 3s 288ms/step - loss: 3.5968 - pitch_loss: 2.2287 - duration_loss: 1.3681 - val_loss: 4.0069 - val_pitch_loss: 2.6373 - val_duration_loss: 1.